In [7]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import sklearn
import sklearn.metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD

import re
import nltk
import unicodedata
from nltk import PunktSentenceTokenizer
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import transformers
import gensim.downloader

import xfeat
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold
import lightgbm as lgb
import gensim.downloader
from gensim.models import KeyedVectors

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [3]:
def concat_train_test(train, test):
    all_df = pd.merge(train, test, how="outer")
    all_df["data_type"] = ""
    for n in range(len(all_df)):
        all_df["data_type"][n] = "test" if np.isnan(all_df["state"][n]) else "train"
    return all_df

In [4]:
all_df = concat_train_test(train, test)
all_df

<ipython-input-3-685f4a3720c8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_df["data_type"][n] = "test" if np.isnan(all_df["state"][n]) else "train"


,id,goal,country,duration,category1,category2,html_content,state,data_type
0,0,4001-5000,CH,29,publishing,young adult,"<div class=""contents""><div><span class=""bold"">...",0.0,train
1,1,3001-4000,NL,34,fashion,ready-to-wear,"<div class=""contents""><div><h1 class=""page-anc...",0.0,train
2,2,19001-20000,US,30,food,spaces,"<div class=""contents""><div><p> As our society ...",0.0,train
3,3,2001-3000,US,41,technology,3d printing,"<div class=""contents""><div><p>My name is Donal...",0.0,train
4,4,2001-3000,GB,29,technology,diy electronics,"<div class=""contents""><div><div class=""templat...",1.0,train
...,...,...,...,...,...,...,...,...,...
21084,21084,9001-10000,US,30,food,drinks,"<div class=""contents""><div><p>Its time to get ...",NaN,test
21085,21085,1-1000,US,29,food,small batch,"<div class=""contents""><div><p>I have been roas...",NaN,test
21086,21086,1001-2000,US,27,crafts,pottery,"<div class=""contents""><div><p> I have ...",NaN,test
21087,21087,2001-3000,US,30,design,graphic design,"<div class=""contents""><div><h1 class=""page-anc...",NaN,test


In [5]:
def cleanup_spaces(html):
    sbd = PunktSentenceTokenizer()
    html_no_spaces = " ".join(sbd.sentences_from_text(html, realign_boundaries=True))
    return re.sub(r'\s+', " ", html_no_spaces)


def cleanup_japanese(html):
    return re.sub('[ぁ-ん ァ-ン 一-龥]', '', html)


def number_of_sentences(html):
    sbd = PunktSentenceTokenizer()
    return len(sbd.sentences_from_text(html, realign_boundaries=True))

In [6]:
def extract_each_paragraphs(html_text):
    paragraphs = []
    para_list = html_text.split("<p>")
    if not "<p>" in html_text[0:4]:
        para_list = para_list[1:]
    for para in para_list:
        if "</p>" in para:
            para_new = para.split("</p>")[0]
        else:
            para_new = ""
            
        #それでも残っているhtmlタグを取り除く(要らないかも)
        para_new = get_text(para_new)
        #para_new = remove_nobreak_space(para_new)
        
        paragraphs.append(para_new)
    try:
        paragraphs.remove("")
    except:
        pass
    try:
        paragraphs.remove(" ")
    except:
        pass
    
    return paragraphs

def number_of_chars(html):
    return len(" ".join(html.split()))

def number_of_words(html):
    return len(html.split())

def number_of_excmark(html):
    return html.count("!")

def number_of_questmark(html):
    return html.count("?")

def number_of_punctuation(html):
    return sum(html.count(w) for w in ".,;:")

def number_of_symbols(html):
    return sum(html.count(w) for w in "*$%&")

def number_of_unique_words(html):
    return len(set(w for w in html.split()))

In [7]:
def is_video_exist(n_video):
    is_video_exist = 1 if n_video != 0 else 0
    return is_video_exist

In [8]:
def compile_html_tag(html):
    html_tags = ["a", "div", "!", "span", "img", "button", "video", "figure",
                 "figcaption", "h1","h1 ", "h2", "h3", "h4", "h5", "h6", "polygon",
                 "iframe", "style", "svg", "use", "time", "source", "li",
                 "ul", "track", "embed", "input", "param",
                ]
    html = html.replace(u'\xa0', u' ')
    html = html.replace(u'\xc2', u' ')
    spaces = re.compile(r'\s+')
    html = spaces.sub(" ", html)
    url = re.compile(r'https?://\S+|www\.\S+')
    html = url.sub("<URL>", html)
    
    for tag in html_tags:
        compiler = re.compile(r"<{}.*?>".format(tag))
        html = compiler.sub(f"<{tag}>", html)
        
    return html

In [9]:
def extract_html_tag(html_text):
    tag_list = re.findall(r"<.*?>", html_text)
    return tag_list

In [10]:
def make_tag_set(df):
    tags = set()
    for html_text in df.html_content:
        compiled_html_text = compile_html_tag(html_text)
        tags |= set(extract_html_tag(compiled_html_text))
    return tags

def count_n_tag(html_tag):
    html, tag = html_tag
    return html.count(tag)

def add_number_of_tag(df_origin):
    df = df_origin.copy()
    tag_set = make_tag_set(df)
    for tag in tag_set:
        df[f"number_of_{tag}"] = list(map(count_n_tag, zip(df["html_compiled"], [tag]*len(df))))
    return df

In [11]:
ALL_TAGS = make_tag_set(all_df)
def get_text(html):
    html_tags = ALL_TAGS
    for tag in html_tags:
        html = re.sub(tag, "", html)
    html = cleanup_spaces(html)
    return html

In [12]:
def col_is_nonzero(col):
    return 0 if col == 0 else 1

In [13]:
def normalize(text):
    #text = cleanup_japanese(text)
    #text = cleanup_spaces(text)
    normalized_text = normalize_unicode(text)
    normalized_text = normalize_number(normalized_text)
    normalized_text = lower_text(normalized_text)
    return normalized_text


def lower_text(text):
    return text.lower()


def normalize_unicode(text, form='NFKC'):
    normalized_text = unicodedata.normalize(form, text)
    return normalized_text


def normalize_number(text):
    # 連続した数字を除去
    replaced_text = re.sub(r'\d+', '', text)
    return replaced_text

In [14]:
def apply_nlp_preprofuncs(df_origin):
    df = df_origin.copy()
    preprofuncs = [
        (compile_html_tag, "html_compiled", "html_content"),
        (get_text, "html_raw", "html_compiled"),
        (number_of_chars, "number_of_chars", "html_raw"),
        (number_of_words, "number_of_words", "html_raw"),
        (number_of_sentences, "number_of_sentences", "html_raw"),
        (number_of_excmark, "number_of_excmark", "html_raw"),
        (number_of_questmark, "number_of_questmark", "html_raw"),
        (number_of_punctuation, "number_of_punctuation", "html_raw"),
        (number_of_symbols, "number_of_symbols", "html_raw"),
        (number_of_unique_words, "number_of_unique_words", "html_raw"),
    ]
    for func, col_name, target in tqdm(preprofuncs):
        df[col_name] = list(map(func, df[target]))
    df = add_number_of_tag(df)
    
    df = df.drop(["html_content", "html_compiled", "html_raw"], axis=1)
    return df

In [15]:
def apply_nlp_preprofuncs_for_bert(df_origin):
    df = df_origin.copy()
    preprofuncs = [
        (compile_html_tag, "html_compiled", "html_content"),
        (get_text, "html_raw", "html_compiled"),
        (normalize, "normalized_html_raw", "html_raw")
    ]
    for func, col_name, target in tqdm(preprofuncs):
        df[col_name] = list(map(func, df[target]))
    
    return df

In [16]:
nlp_preprocessed_df = apply_nlp_preprofuncs(all_df)
text_type_df = apply_nlp_preprofuncs_for_bert(all_df)
text_type_df.head()

100%|██████████| 3/3 [00:18<00:00,  6.04s/it]


,id,goal,country,duration,category1,category2,html_content,state,data_type,html_compiled,html_raw,normalized_html_raw
0,0,4001-5000,CH,29,publishing,young adult,"<div class=""contents""><div><span class=""bold"">...",0.0,train,<div><div><span>Mark Saggia</span> is an Itali...,Mark Saggia is an Italian writer who emigrated...,mark saggia is an italian writer who emigrated...
1,1,3001-4000,NL,34,fashion,ready-to-wear,"<div class=""contents""><div><h1 class=""page-anc...",0.0,train,"<div><div><h1>Hello, I am Augustinas. I am a g...","Hello, I am Augustinas. I am a graphic designe...","hello, i am augustinas. i am a graphic designe..."
2,2,19001-20000,US,30,food,spaces,"<div class=""contents""><div><p> As our society ...",0.0,train,<div><div><p> As our society begins to wake up...,As our society begins to wake up from the han...,as our society begins to wake up from the han...
3,3,2001-3000,US,41,technology,3d printing,"<div class=""contents""><div><p>My name is Donal...",0.0,train,<div><div><p>My name is Donald Osborne and I a...,My name is Donald Osborne and I am an entrepre...,my name is donald osborne and i am an entrepre...
4,4,2001-3000,GB,29,technology,diy electronics,"<div class=""contents""><div><div class=""templat...",1.0,train,<div><div><div> <figure> <img> </figure> </div...,"We all love to play, don't we! No matter the ...","we all love to play, don't we! no matter the ..."


In [17]:
nlp_preprocessed_df.to_csv('data/nlp_prepared_df.csv', index=False)
text_type_df.to_csv('data/nlp_prepared_for_bert.csv', index=False)

In [5]:
def make_tfidf_svd(df, target_col, stop_words="english", max_gram=2, num_features_tfidf=2000, num_features_svd=128):
    vectorizer = TfidfVectorizer(stop_words="english",
                                 ngram_range=(1,max_gram),
                                max_features=num_features_tfidf,
                                )
    transformer = TruncatedSVD(n_components=num_features_svd)
    
    X = vectorizer.fit_transform(df[target_col])
    matrix = transformer.fit_transform(X)
    
    columns = [f"tfidf_svd_{target_col}_{dim}" for dim in range(num_features_svd)]
    feature_df = pd.DataFrame(matrix, columns=columns)
    ids = [n for n in range(len(df))]
    feature_df["id"] = ids
    return feature_df

In [9]:
text_type_df = pd.read_csv("data/nlp_prepared_for_bert.csv")
text_type_df.html_raw = text_type_df.html_raw.fillna("").astype(str)
tfidf_svd_raw_128_2gram = make_tfidf_svd(text_type_df, "html_raw", stop_words="english", max_gram=2, num_features_tfidf=None, num_features_svd=128)
tfidf_svd_raw_128_2gram.to_csv("data/tfidf_svd_raw_128_2gram.csv", index=False)

In [13]:
def extract_most_freq_word(df_origin):
    df = df_origin.copy()
    
    num_features = 1
    vectorizer = CountVectorizer(stop_words="english",
                                 ngram_range=(1,1),
                                max_features=num_features,)
    mfw_dict = {}
    for n in range(len(df)):
        try:
            vectorizer.fit_transform([df.normalized_html_raw.iloc[n]])
            feature = vectorizer.get_feature_names()[0]
        except:
            # no text
            feature = ""
        mfw_dict[f"{n}"] = feature
    mfw_df = pd.DataFrame({"id": mfw_dict.keys(), "mfw": mfw_dict.values()})
    
    
    num_features = 2
    vectorizer = CountVectorizer(stop_words="english",
                                 ngram_range=(1,1),
                                max_features=num_features,)
    sfw_dict = {}
    for n in range(len(df)):
        try:
            vectorizer.fit_transform([df.normalized_html_raw.iloc[n]])
            feature = list(vectorizer.get_feature_names())
            feature.remove(mfw_df["mfw"].iloc[n])
            feature = feature[0]
        except:
            # no text
            feature = ""
        sfw_dict[f"{n}"] = feature     
    mfw_df["sfw"] =  sfw_dict.values()
    
    return mfw_df

# you can use any type 
glove_short_vectors = gensim.downloader.load('glove-twitter-25')
def word_to_vector2(words):
    try:
        vector = glove_short_vectors[words[0]]
    except:
        try:
            vector = glove_short_vectors[words[1]]
        except:
            vector = np.zeros(25)
    return vector


def add_wordvec2(df_origin, columns):
    df = df_origin.copy()
    col1, col2 = columns
    df[[f"mfw_{n}" for n in range(25)]] = list(map(word_to_vector2, zip(df[col1], df[col2])))
    return df

In [12]:
def extract_most_informative_word(df_origin):
    df = df_origin.copy()
    
    num_features = 5000
    vectorizer = TfidfVectorizer(stop_words="english",
                                 ngram_range=(1,1),
                                max_features=num_features,)
    X = vectorizer.fit_transform(df.normalized_html_raw)
    matrix = vectorizer.transform(df.normalized_html_raw)
    feature_name = vectorizer.get_feature_names()
    
    miws = []
    ids = df["id"]
    for n in tqdm(range(len(df))):
        miw_column = np.argmax(matrix[n])
        miw = feature_name[miw_column]
        miws.append(miw)
    miw_df = pd.DataFrame({"id": ids, "miw": miws})
    return miw_df


def word_to_vector(words):
    try:
        vector = glove_short_vectors[words]
    except:
        print(words)
        vector = np.zeros(25)
    return vector


def add_wordvec(df_origin):
    df = df_origin.copy()
    df[[f"miw_{n}" for n in range(25)]] = list(map(word_to_vector, df["miw"]))
    df = df.drop("miw", axis=1)
    return df

In [14]:
mfw_df = extract_most_freq_word(text_type_df)
mfw_df = add_wordvec2(mfw_df, ("mfw", "sfw"))

In [15]:
mfw_df = mfw_df.drop(["mfw", "sfw"], axis=1)
mfw_df.to_csv('data/mfw.csv', index=False)
mfw_df

,id,mfw_0,mfw_1,mfw_2,mfw_3,mfw_4,mfw_5,mfw_6,mfw_7,mfw_8,...,mfw_15,mfw_16,mfw_17,mfw_18,mfw_19,mfw_20,mfw_21,mfw_22,mfw_23,mfw_24
0,0,1.146000,0.329100,0.268780,-1.394500,-0.300440,0.77901,1.35370,0.37393,0.504780,...,0.102870,-0.17618,-1.28810,-0.598010,0.26131,-1.26190,0.392020,0.593090,-0.55232,0.005087
1,1,0.615310,0.408950,-0.342240,0.210940,-0.046403,0.29744,1.94850,-0.91247,-0.857720,...,0.352070,0.16624,-0.91364,0.693300,-0.65035,0.77353,-0.087782,-0.432280,-0.99982,-0.680770
2,2,0.077850,0.065545,-0.349580,-0.378050,0.873780,0.24333,1.49570,-0.56260,0.070204,...,0.355000,0.16465,0.32006,-0.045544,-0.31911,-1.01380,-0.054359,0.200740,0.32061,-0.455840
3,3,-0.551330,-0.005450,-0.795840,-0.384020,1.222400,-0.10110,-0.16260,-1.81530,0.835090,...,-0.910800,0.69415,-0.32473,1.184100,0.45249,-1.03210,-1.265900,-0.740850,0.26441,-0.497070
4,4,-0.258740,-0.494780,0.327090,-0.045509,0.255290,1.34700,1.20420,-0.70687,-0.308090,...,-0.249380,1.02640,-0.10366,-0.379820,-0.82619,-0.20513,0.572080,0.434970,0.32462,-0.332930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21084,21084,0.024986,0.021567,-0.330700,-0.389370,0.978020,0.62207,-0.56160,0.37704,0.520450,...,0.508290,0.56018,0.79872,0.235690,0.99819,-0.85112,-0.637180,-0.144610,0.18872,0.382730
21085,21085,-1.453200,0.255940,0.081679,0.490810,-0.059128,0.63656,1.45660,-1.06070,0.479030,...,-0.509060,1.24320,-0.54121,0.193750,0.35703,-0.98850,-0.693350,-0.339980,0.71827,1.070400
21086,21086,-0.495150,1.283500,-1.236000,0.708940,0.295160,-0.48593,0.55181,-1.05530,-0.291370,...,0.068498,0.51722,-0.20461,0.794370,-0.69678,-0.94534,-0.336830,-0.805960,0.54258,-0.736610
21087,21087,-0.177000,0.432000,0.213910,-0.295150,0.473440,-0.38013,1.60180,0.56238,0.404780,...,-0.713360,0.50856,-0.25976,-0.479320,0.60913,-1.00300,-1.022500,-0.099247,-0.58948,-0.424680


In [30]:
miw_df = extract_most_informative_word(text_type_df)
miw_df = add_wordvec(miw_df)
miw_df.to_csv('data/miw.csv', index=False)
miw_df

100%|██████████| 21089/21089 [00:02<00:00, 7258.53it/s]


このコンテンツを表示するにはhtml対応のブラウザが必要です
このコンテンツを表示するにはhtml対応のブラウザが必要です
このコンテンツを表示するにはhtml対応のブラウザが必要です
このコンテンツを表示するにはhtml対応のブラウザが必要です
このコンテンツを表示するにはhtml対応のブラウザが必要です
このコンテンツを表示するにはhtml対応のブラウザが必要です
このコンテンツを表示するにはhtml対応のブラウザが必要です
このコンテンツを表示するにはhtml対応のブラウザが必要です
このコンテンツを表示するにはhtml対応のブラウザが必要です
このコンテンツを表示するにはhtml対応のブラウザが必要です
このコンテンツを表示するにはhtml対応のブラウザが必要です
このコンテンツを表示するにはhtml対応のブラウザが必要です
このコンテンツを表示するにはhtml対応のブラウザが必要です
このコンテンツを表示するにはhtml対応のブラウザが必要です
このコンテンツを表示するにはhtml対応のブラウザが必要です
このコンテンツを表示するにはhtml対応のブラウザが必要です
このコンテンツを表示するにはhtml対応のブラウザが必要です
このコンテンツを表示するにはhtml対応のブラウザが必要です
このコンテンツを表示するにはhtml対応のブラウザが必要です
このコンテンツを表示するにはhtml対応のブラウザが必要です
このコンテンツを表示するにはhtml対応のブラウザが必要です
このコンテンツを表示するにはhtml対応のブラウザが必要です
このコンテンツを表示するにはhtml対応のブラウザが必要です
このコンテンツを表示するにはhtml対応のブラウザが必要です
このコンテンツを表示するにはhtml対応のブラウザが必要です
このコンテンツを表示するにはhtml対応のブラウザが必要です
このコンテンツを表示するにはhtml対応のブラウザが必要です
このコンテンツを表示するにはhtml対応のブラウザが必要です
このコンテンツを表示するにはhtml対応のブラウザが必要です
このコンテンツを表示するにはhtml対応のブラウザが必要です
このコンテンツを表示するにはhtml対応のブラウザが必要です
このコンテンツを表示するにはhtml対応のブラウザが必要です
このコンテンツを

,id,miw_0,miw_1,miw_2,miw_3,miw_4,miw_5,miw_6,miw_7,miw_8,...,miw_15,miw_16,miw_17,miw_18,miw_19,miw_20,miw_21,miw_22,miw_23,miw_24
0,0,1.14600,0.329100,0.268780,-1.394500,-0.300440,0.77901,1.35370,0.373930,0.504780,...,0.102870,-0.17618,-1.288100,-0.598010,0.261310,-1.261900,0.392020,0.593090,-0.552320,0.005087
1,1,-0.73473,0.120720,0.607640,-0.504740,-0.057365,0.52869,0.59001,-0.237530,1.140600,...,-0.034995,0.10252,0.021756,-0.308370,0.430040,0.002524,-0.504040,0.230500,-1.449700,0.589240
2,2,0.07785,0.065545,-0.349580,-0.378050,0.873780,0.24333,1.49570,-0.562600,0.070204,...,0.355000,0.16465,0.320060,-0.045544,-0.319110,-1.013800,-0.054359,0.200740,0.320610,-0.455840
3,3,0.70687,0.277370,0.291350,-0.326550,0.538730,-0.65520,1.35720,-0.252000,-0.316420,...,-0.149920,0.71925,-0.162080,0.623380,0.380480,-0.045843,0.478960,-0.523910,-0.798070,0.278350
4,4,0.30963,-0.206810,-0.041067,-0.149900,0.843800,0.64058,-0.54087,0.055304,1.194000,...,0.394530,0.79590,0.501160,0.056856,1.026600,-0.793950,-0.804790,1.111700,0.013755,-0.244400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21084,21084,-0.52575,-0.229000,0.317210,1.120200,-0.113580,0.43399,0.94626,-1.406300,0.277020,...,-0.432380,1.08140,-0.507060,0.429500,0.373860,-0.675640,0.441460,0.082967,1.074900,0.846170
21085,21085,-1.45320,0.255940,0.081679,0.490810,-0.059128,0.63656,1.45660,-1.060700,0.479030,...,-0.509060,1.24320,-0.541210,0.193750,0.357030,-0.988500,-0.693350,-0.339980,0.718270,1.070400
21086,21086,-0.49515,1.283500,-1.236000,0.708940,0.295160,-0.48593,0.55181,-1.055300,-0.291370,...,0.068498,0.51722,-0.204610,0.794370,-0.696780,-0.945340,-0.336830,-0.805960,0.542580,-0.736610
21087,21087,0.85912,-0.573080,1.341200,0.021117,0.516680,-0.37890,0.11455,-0.635790,-0.581480,...,1.098900,0.28815,-0.689680,0.233150,0.089295,-0.993910,-0.822220,-0.341300,-0.357680,-0.550620
